In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime
import time

In [2]:
headers = {
    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/79.0.3945.88 Safari/537.36'}

In [3]:
def pull_store_95():
    page = requests.get(r'http://www.95mtg.com/buylist/', headers=headers)

    soup = BeautifulSoup(page.content, 'html.parser')

    page_1 = list(soup.children)[2]

    page_2 = list(page_1.children)[1]

    body = list(page_2.children)[7]

    body_1 = list(body.children)[0]

    string = str(body_1)

    cut_before = string.split('var card_master = {"')[1]

    cut_after = cut_before.split('};\n	var half_it = width_master/3;')[0]

    cards = cut_after.split('},"')

    lists = []
    for card in cards:
        name = card.split('card_name":"')[1].split('","price')[0]
        price = float(card.split('price":"$')[1].split('","quan')[0])
        lists.append([name, price])

    data = pd.DataFrame(lists, columns=['name', 'price'])

    data['uuid'] = np.nan

    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")
    cur = conn.cursor()

    new_data = pd.DataFrame(columns=['uuid', 'price'])

    for index, row in data.iterrows():
        # To prevent a bug where single quotes in the name don't work
        if "'" in row['name']:
            cardName = row['name'].replace("'", "''")
        else:
            cardName = row['name']
        SQL = 'SELECT uuid FROM CardInfo WHERE cardName=\'' + cardName + '\';'

        uuids = pd.read_sql_query(SQL, conn)
        uuids['price'] = row.price

        new_data = new_data.append(uuids)
        new_data['date'] = datetime.now()
    
    t_df = tuple(new_data[['uuid', 'date', 'price']].itertuples(index=False, name=None))
    execute_values(cur, "INSERT INTO ninetybuydata (uuid, ninetydate, ninetybuyprice) VALUES %s", t_df)
    conn.commit()

    cur.close()
    conn.close()

In [4]:
def main():
    pull_store_95()

In [5]:
if __name__ == '__main__':
    main()